In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import re
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
bbc_df = pd.read_csv("bbc_news.csv")
bbc_df.head()

,Unnamed: 0,index,title,pubDate,guid,link,description
0,0,6684,Can I refuse to work?,"Wed, 10 Aug 2022 15:46:18 GMT",https://www.bbc.co.uk/news/business-62147992,https://www.bbc.co.uk/news/business-62147992?a...,With much of the UK enduring another period of...
1,1,9267,'Liz Truss the Brief?' World reacts to UK poli...,"Mon, 17 Oct 2022 11:35:12 GMT",https://www.bbc.co.uk/news/world-63285480,https://www.bbc.co.uk/news/world-63285480?at_m...,The UK's political chaos has been watched arou...
2,2,7387,Rationing energy is nothing new for off-grid c...,"Wed, 31 Aug 2022 05:20:18 GMT",https://www.bbc.co.uk/news/uk-scotland-highlan...,https://www.bbc.co.uk/news/uk-scotland-highlan...,Scoraig in the north west Highlands has long h...
3,3,767,The hunt for superyachts of sanctioned Russian...,"Tue, 22 Mar 2022 14:37:01 GMT",https://www.bbc.co.uk/news/60739336,https://www.bbc.co.uk/news/60739336?at_medium=...,"Wealthy Russians sanctioned by the US, EU and ..."
4,4,3712,Platinum Jubilee: 70 years of the Queen in 70 ...,"Wed, 01 Jun 2022 23:17:33 GMT",https://www.bbc.co.uk/news/uk-61660128,https://www.bbc.co.uk/news/uk-61660128?at_medi...,A quick look back at the Queen's 70 years on t...


In [3]:
bbc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1000 non-null   int64 
 1   index        1000 non-null   int64 
 2   title        1000 non-null   object
 3   pubDate      1000 non-null   object
 4   guid         1000 non-null   object
 5   link         1000 non-null   object
 6   description  1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [5]:
titles = pd.DataFrame(bbc_df["title"])
titles

,title
0,Can I refuse to work?
1,'Liz Truss the Brief?' World reacts to UK poli...
2,Rationing energy is nothing new for off-grid c...
3,The hunt for superyachts of sanctioned Russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...
...,...
995,Dominic Raab: Third senior civil servant gives...
996,Highlights: Radacanu beats Uytvanck
997,In pictures: Mountain bikers descend snowy peak
998,"Companies must help cut living costs, says new..."


## Preprocessing 

### Clean Data

In [7]:
#lowercase
titles["lowercase"] = titles["title"].str.lower()
titles.head()

,title,lowercase
0,Can I refuse to work?,can i refuse to work?
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...


In [16]:
#removing stopwords
en_stopwords = stopwords.words("english")

titles["no_stopwords"] = titles["lowercase"].apply(lambda x: ' '.join([word for word in x.split() if word not in (en_stopwords)]))
titles.head()

,title,lowercase,no_stopwords,no_stopwords_no_punc,tokens,tokens_clean,tokens_clean_lemmatized
0,Can I refuse to work?,can i refuse to work?,refuse work?,refusework,"[Can, I, refuse, to, work, ?]",[refusework],[refusework]
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liztrussbriefworldreactsukpoliticalturmoil,"['Liz, Truss, the, Brief, ?, ', World, reacts,...",[liztrussbriefworldreactsukpoliticalturmoil],[liztrussbriefworldreactsukpoliticalturmoil]
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationingenergynothingnewoffgridcommunity,"[Rationing, energy, is, nothing, new, for, off...",[rationingenergynothingnewoffgridcommunity],[rationingenergynothingnewoffgridcommunity]
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,huntsuperyachtssanctionedrussianoligarchs,"[The, hunt, for, superyachts, of, sanctioned, ...",[huntsuperyachtssanctionedrussianoligarchs],[huntsuperyachtssanctionedrussianoligarchs]
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinumjubilee70yearsqueen70seconds,"[Platinum, Jubilee, :, 70, years, of, the, Que...",[platinumjubilee70yearsqueen70seconds],[platinumjubilee70yearsqueen70seconds]


In [17]:
#removing ponctuation
titles["no_stopwords_no_punc"] = titles.apply(lambda x: re.sub(r"([^\w\s])","",x["no_stopwords"]),axis=1)
titles.head()

,title,lowercase,no_stopwords,no_stopwords_no_punc,tokens,tokens_clean,tokens_clean_lemmatized
0,Can I refuse to work?,can i refuse to work?,refuse work?,refuse work,"[Can, I, refuse, to, work, ?]",[refusework],[refusework]
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil,"['Liz, Truss, the, Brief, ?, ', World, reacts,...",[liztrussbriefworldreactsukpoliticalturmoil],[liztrussbriefworldreactsukpoliticalturmoil]
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community,"[Rationing, energy, is, nothing, new, for, off...",[rationingenergynothingnewoffgridcommunity],[rationingenergynothingnewoffgridcommunity]
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs,"[The, hunt, for, superyachts, of, sanctioned, ...",[huntsuperyachtssanctionedrussianoligarchs],[huntsuperyachtssanctionedrussianoligarchs]
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds,"[Platinum, Jubilee, :, 70, years, of, the, Que...",[platinumjubilee70yearsqueen70seconds],[platinumjubilee70yearsqueen70seconds]


In [18]:
#Tokenization
titles["tokens"] = titles.apply(lambda x: word_tokenize(x["title"]), axis=1)
titles["tokens_clean"] = titles.apply(lambda x: word_tokenize(x["no_stopwords_no_punc"]), axis=1)
titles.head()

,title,lowercase,no_stopwords,no_stopwords_no_punc,tokens,tokens_clean,tokens_clean_lemmatized
0,Can I refuse to work?,can i refuse to work?,refuse work?,refuse work,"[Can, I, refuse, to, work, ?]","[refuse, work]",[refusework]
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil,"['Liz, Truss, the, Brief, ?, ', World, reacts,...","[liz, truss, brief, world, reacts, uk, politic...",[liztrussbriefworldreactsukpoliticalturmoil]
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community,"[Rationing, energy, is, nothing, new, for, off...","[rationing, energy, nothing, new, offgrid, com...",[rationingenergynothingnewoffgridcommunity]
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs,"[The, hunt, for, superyachts, of, sanctioned, ...","[hunt, superyachts, sanctioned, russian, oliga...",[huntsuperyachtssanctionedrussianoligarchs]
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds,"[Platinum, Jubilee, :, 70, years, of, the, Que...","[platinum, jubilee, 70, years, queen, 70, seco...",[platinumjubilee70yearsqueen70seconds]


In [19]:
#Lemmatization

lemmatizer = WordNetLemmatizer()
titles["tokens_clean_lemmatized"] = titles["tokens_clean"].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
titles.head()

,title,lowercase,no_stopwords,no_stopwords_no_punc,tokens,tokens_clean,tokens_clean_lemmatized
0,Can I refuse to work?,can i refuse to work?,refuse work?,refuse work,"[Can, I, refuse, to, work, ?]","[refuse, work]","[refuse, work]"
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil,"['Liz, Truss, the, Brief, ?, ', World, reacts,...","[liz, truss, brief, world, reacts, uk, politic...","[liz, truss, brief, world, reacts, uk, politic..."
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community,"[Rationing, energy, is, nothing, new, for, off...","[rationing, energy, nothing, new, offgrid, com...","[rationing, energy, nothing, new, offgrid, com..."
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs,"[The, hunt, for, superyachts, of, sanctioned, ...","[hunt, superyachts, sanctioned, russian, oliga...","[hunt, superyachts, sanctioned, russian, oliga..."
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds,"[Platinum, Jubilee, :, 70, years, of, the, Que...","[platinum, jubilee, 70, years, queen, 70, seco...","[platinum, jubilee, 70, year, queen, 70, second]"


In [21]:
# get all tokens in one list

tokens_list = sum(titles["tokens"],[])
tokens_clean_list = sum(titles["tokens_clean"],[])
print("List of all tokens : ", tokens_list)


List of all tokens :  ['Can', 'I', 'refuse', 'to', 'work', '?', "'Liz", 'Truss', 'the', 'Brief', '?', "'", 'World', 'reacts', 'to', 'UK', 'political', 'turmoil', 'Rationing', 'energy', 'is', 'nothing', 'new', 'for', 'off-grid', 'community', 'The', 'hunt', 'for', 'superyachts', 'of', 'sanctioned', 'Russian', 'oligarchs', 'Platinum', 'Jubilee', ':', '70', 'years', 'of', 'the', 'Queen', 'in', '70', 'seconds', 'Red', 'Bull', 'found', 'guilty', 'of', 'breaking', 'Formula', '1', "'s", 'budget', 'cap', 'World', 'Triathlon', 'Championship', 'Series', ':', 'Flora', 'Duffy', 'beats', 'Georgia', 'Taylor-Brown', 'to', 'women', "'s", 'title', 'Terry', 'Hall', ':', 'Coventry', 'scooter', 'ride-out', 'pays', 'tribute', 'to', 'singer', 'Post', 'Office', 'and', 'Fujitsu', 'to', 'face', 'inquiry', 'over', 'Horizon', 'scandal', "'Pavement", 'parking', 'frightens', 'me', "'", 'UK', 'interest', 'rates', ':', 'How', 'will', 'the', 'rise', 'affect', 'you', 'and', 'how', 'high', 'could', 'it', 'go', '?', 'The

In [22]:
print("List of all cleaned tokens : ", tokens_clean_list)

List of all cleaned tokens :  ['refuse', 'work', 'liz', 'truss', 'brief', 'world', 'reacts', 'uk', 'political', 'turmoil', 'rationing', 'energy', 'nothing', 'new', 'offgrid', 'community', 'hunt', 'superyachts', 'sanctioned', 'russian', 'oligarchs', 'platinum', 'jubilee', '70', 'years', 'queen', '70', 'seconds', 'red', 'bull', 'found', 'guilty', 'breaking', 'formula', '1s', 'budget', 'cap', 'world', 'triathlon', 'championship', 'series', 'flora', 'duffy', 'beats', 'georgia', 'taylorbrown', 'womens', 'title', 'terry', 'hall', 'coventry', 'scooter', 'rideout', 'pays', 'tribute', 'singer', 'post', 'office', 'fujitsu', 'face', 'inquiry', 'horizon', 'scandal', 'pavement', 'parking', 'frightens', 'me', 'uk', 'interest', 'rates', 'rise', 'affect', 'high', 'could', 'go', 'stayed', 'storm', 'happens', 'now', 'six', 'nations', 'scotlands', 'best', 'since', '99', 'beat', 'best', 'ireland', 'ever', 'long', 'liz', 'truss', 'survive', 'prime', 'minister', 'platinum', 'jubilee', 'beacons', 'light', 'a

## POS tagging

In [23]:
nlp = spacy.load("en_core_web_sm")

In [24]:
spacy_doc = nlp(' '.join(tokens_list))
spacy_doc

Can I refuse to work ? 'Liz Truss the Brief ? ' World reacts to UK political turmoil Rationing energy is nothing new for off-grid community The hunt for superyachts of sanctioned Russian oligarchs Platinum Jubilee : 70 years of the Queen in 70 seconds Red Bull found guilty of breaking Formula 1 's budget cap World Triathlon Championship Series : Flora Duffy beats Georgia Taylor-Brown to women 's title Terry Hall : Coventry scooter ride-out pays tribute to singer Post Office and Fujitsu to face inquiry over Horizon scandal 'Pavement parking frightens me ' UK interest rates : How will the rise affect you and how high could it go ? They stayed for the storm - what happens now ? Six Nations : Can Scotland 's best since '99 beat best Ireland ever ? How long can Liz Truss survive as prime minister ? Platinum Jubilee : Beacons light across the globe to celebrate Queen 's reign Top student loan interest rate cut by 5 % in England Russians said they ’ d take my baby : A medic ’ s story Week in 

In [31]:
pos_df = pd.DataFrame(columns=["token","pos_tag"])

# loop over each token in our spacy document and extract its corresponding pos tag

for token in spacy_doc :
    pos_df = pd.concat([pos_df,
                       pd.DataFrame.from_records([{'token': token.text, 'pos_tag': token.pos_}])], ignore_index=True)
        


In [32]:
pos_df

,token,pos_tag
0,Can,AUX
1,I,PRON
2,refuse,VERB
3,to,PART
4,work,VERB
...,...,...
11742,sale,NOUN
11743,scams,NOUN
11744,",",PUNCT
11745,consumers,NOUN


### create a column for token frequency counts

In [33]:
pos_df_counts = pos_df.groupby(["token","pos_tag"]).size().reset_index(name="counts").sort_values(by="counts",ascending=False)
pos_df_counts

,token,pos_tag,counts
95,:,PUNCT,543
8,',PUNCT,300
2897,in,ADP,187
4082,to,PART,175
3268,of,ADP,172
...,...,...,...
2304,crumbling,VERB,1
2305,crunch,PROPN,1
827,Jarrod,PROPN,1
826,Japanese,ADJ,1


In [34]:
#get most common nouns

nouns = pos_df_counts[pos_df_counts.pos_tag =="NOUN"].sort_values(by="counts", ascending=False)
nouns

,token,pos_tag,counts
4267,war,NOUN,35
3552,record,NOUN,15
3416,police,NOUN,14
4356,year,NOUN,14
4316,win,NOUN,14
...,...,...,...
3980,sweets,NOUN,1
3982,swoop,NOUN,1
3984,syndrome,NOUN,1
4011,tea,NOUN,1


In [38]:
#get most common verbs

verbs = pos_df_counts[pos_df_counts.pos_tag =="VERB"].sort_values(by="counts", ascending=False)
verbs

,token,pos_tag,counts
3687,says,VERB,30
9,',VERB,14
2670,found,VERB,13
4317,win,VERB,12
4324,wins,VERB,10
...,...,...,...
4276,watch,VERB,1
4271,warning,VERB,1
4304,whipped,VERB,1
4287,wearing,VERB,1


In [39]:
#get most common adjectives
adj = pos_df_counts[pos_df_counts.pos_tag =="ADJ"].sort_values(by="counts", ascending=False)
adj

,token,pos_tag,counts
3244,new,ADJ,28
1400,Russian,ADJ,21
2606,final,ADJ,16
19,-,ADJ,14
2625,first,ADJ,12
...,...,...,...
4188,unrecognisable,ADJ,1
4191,unusual,ADJ,1
4202,urinal,ADJ,1
4206,uterine,ADJ,1


## NER : Name Entity Recognition

In [44]:
ner_df = pd.DataFrame(columns=["token","ner_tag"])

for token in spacy_doc.ents : 
    if pd.isna(token.label_) is False :
       ner_df = pd.concat([ner_df, pd.DataFrame.from_records([{"token": token.text, "ner_tag": token.label_}])], ignore_index=True)

ner_df

,token,ner_tag
0,Liz Truss,PERSON
1,UK,GPE
2,Rationing,PRODUCT
3,superyachts,CARDINAL
4,Russian,NORP
...,...,...
1665,Frenkie de Jong,PERSON
1666,Manchester United,PERSON
1667,Barcelona,GPE
1668,Dominic Raab,PERSON


In [46]:
# get most common name entities
ner_df_counts = ner_df.groupby(["token","ner_tag"]).size().reset_index(name="counts").sort_values(by="counts",ascending= False)
ner_df_counts

,token,ner_tag,counts
965,Ukraine,GPE,47
955,UK,GPE,36
329,England,GPE,32
819,Russian,NORP,20
957,US,GPE,19
...,...,...,...
405,Georgia Taylor-Brown,PERSON,1
406,Geraint Thomas,PERSON,1
409,Ghislaine Maxwell,PERSON,1
410,Gianluigi Lentini,PERSON,1
